In [1]:
from find_fuzzy_duplicates import find_fuzzy_duplicates


import polars as pl
pl.Config.set_tbl_rows(-1)

polars.config.Config

###### Interne dubletter
# Eksterne dubletter med Oslo/Hamar

In [2]:
###### leser inn Data exportert i fra Bergens Access database
# I fra den tabellen som heter "tblInnmeldte"
df_bergen = (pl.read_excel("tblInnmeldte 2025-06-12.xlsx")
    # Vet ikke hvorfor dette blir gjort, men det er det som skjer i access databasen "qryMedlemmer" -
    # som er utgangspunkt for alle data som vises i Access
    # uten filter finnes 333 medlemmer, mens med filter finnes 284 medlemmer.
    # det er disse som har vært utgangspunkt for medlems opptellig
    .filter(pl.col("POSTNR").is_between(pl.lit("100"), pl.lit("9000"), closed="both"))
)
#print(df_bergen.select("MedlemID", "FORNAVN", "EFTERNAVN", "ADRESSE", "POSTNR").sort(["FORNAVN", "EFTERNAVN"]))

Could not determine dtype for column 43, falling back to string


In [6]:
duplicates_found = find_fuzzy_duplicates(
        df=df_bergen,
        columns=["FORNAVN", "EFTERNAVN"],
        columns_to_view=["Kategori","RegDato" ,"UtmeldtDato", "MedlemID", "ADRESSE", "POSTNR"],
        #columns_to_view=None,
        similarity_threshold=88,
        #blocking_column="POSTNR"
    )
#duplicates_found.select("fuzzy_group","MedlemID", "FORNAVN", "EFTERNAVN", "ADRESSE", "POSTNR")
duplicates_found

fuzzy_group,FORNAVN,EFTERNAVN,Kategori,RegDato,UtmeldtDato,MedlemID,ADRESSE,POSTNR
i64,str,str,str,datetime[ms],date,i64,str,str
0,"""Kari""","""Berger""","""M""",2020-03-20 17:22:29,null,3439,"""Gjerdåsvegen 4""","""5428"""
0,"""Kari Anna""","""Berger""","""M""",2022-08-26 10:09:25,null,4592,"""Nordahl Rolfsensvei 22A""","""5094"""
1,"""Maria Augusta""","""Helland""","""M""",2019-06-13 13:36:59,null,4492,"""Sjkoldvegen 40B""","""5221"""
1,"""Maria""","""Løvland""","""UTM""",2014-09-17 18:03:56,null,3628,"""Presthaugveien 1D""","""5521"""
2,"""Dina Sofie Aeschlimann""","""Isene""","""MB""",2019-09-23 11:20:12,null,3537,"""Tørsdalsvegen 573""","""5574"""
2,"""Oda Sunniva Aeschlimann""","""Isene""","""MB""",2005-01-07 10:13:28,null,2811,"""Tørsdalsvegen 573""","""5574"""
3,"""Lisbeth""","""Sunde""","""V""",2019-06-24 20:24:31,null,4578,"""Løvåsveien 70""","""5145"""
3,"""Lisbeth""","""Sundt""","""V""",2019-04-08 09:14:29,null,2172,"""Gimlebakken 18""","""5052"""
4,"""Amalie Dobrovólny""","""Wolfe""","""MB""",2014-05-21 11:11:23,null,3063,"""Johan Hjortsvei 46""","""5081"""


In [4]:
#df_bergen.sort(["FORNAVN", "EFTERNAVN"])